# Введение в MapReduce модель на Python


In [33]:
from typing import NamedTuple # requires python 3.6+
from typing import Iterator

In [34]:
def MAP(_, row:NamedTuple):
  if (row.gender == 'female'):
    yield (row.age, row)
    
def REDUCE(age:str, rows:Iterator[NamedTuple]):
  sum = 0
  count = 0
  for row in rows:
    sum += row.social_contacts
    count += 1
  if (count > 0):
    yield (age, sum/count)
  else:
    yield (age, 0)

Модель элемента данных

In [35]:
class User(NamedTuple):
  id: int
  age: str
  social_contacts: int
  gender: str

In [36]:
input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

Функция RECORDREADER моделирует чтение элементов с диска или по сети.

In [37]:
def RECORDREADER():
  return [(u.id, u) for u in input_collection]

In [38]:
list(RECORDREADER())

[(0, User(id=0, age=55, social_contacts=20, gender='male')),
 (1, User(id=1, age=25, social_contacts=240, gender='female')),
 (2, User(id=2, age=25, social_contacts=500, gender='female')),
 (3, User(id=3, age=33, social_contacts=800, gender='female'))]

In [39]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

In [40]:
map_output = flatten(map(lambda x: MAP(*x), RECORDREADER()))
map_output = list(map_output) # materialize
map_output

[(25, User(id=1, age=25, social_contacts=240, gender='female')),
 (25, User(id=2, age=25, social_contacts=500, gender='female')),
 (33, User(id=3, age=33, social_contacts=800, gender='female'))]

In [41]:
def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

In [42]:
shuffle_output = groupbykey(map_output)
shuffle_output = list(shuffle_output)
shuffle_output

[(25,
  [User(id=1, age=25, social_contacts=240, gender='female'),
   User(id=2, age=25, social_contacts=500, gender='female')]),
 (33, [User(id=3, age=33, social_contacts=800, gender='female')])]

In [43]:
reduce_output = flatten(map(lambda x: REDUCE(*x), shuffle_output))
reduce_output = list(reduce_output)
reduce_output

[(25, 370.0), (33, 800.0)]

Все действия одним конвейером!

In [44]:
list(flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER()))))))

[(25, 370.0), (33, 800.0)]

# **MapReduce**
Выделим общую для всех пользователей часть системы в отдельную функцию высшего порядка. Это наиболее простая модель MapReduce, без учёта распределённого хранения данных. 

Пользователь для решения своей задачи реализует RECORDREADER, MAP, REDUCE.

In [45]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
  return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

## Спецификация MapReduce



```
f (k1, v1) -> (k2,v2)*
g (k2, v2*) -> (k3,v3)*
 
mapreduce ((k1,v1)*) -> (k3,v3)*
groupby ((k2,v2)*) -> (k2,v2*)*
flatten (e2**) -> e2*
 
mapreduce .map(f).flatten.groupby(k2).map(g).flatten
```




# Примеры

## SQL 

In [46]:
from typing import NamedTuple # requires python 3.6+
from typing import Iterator

class User(NamedTuple):
  id: int
  age: str
  social_contacts: int
  gender: str
    
input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

def MAP(_, row:NamedTuple):
  if (row.gender == 'female'):
    yield (row.age, row)
    
def REDUCE(age:str, rows:Iterator[NamedTuple]):
  sum = 0
  count = 0
  for row in rows:
    sum += row.social_contacts
    count += 1
  if (count > 0):
    yield (age, sum/count)
  else:
    yield (age, 0)
 
def RECORDREADER():
  return [(u.id, u) for u in input_collection]

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(25, 370.0), (33, 800.0)]

## Matrix-Vector multiplication 

In [47]:
from typing import Iterator
import numpy as np

mat = np.ones((5,4))
vec = np.random.rand(4) # in-memory vector in all map tasks

def MAP(coordinates:(int, int), value:int):
  i, j = coordinates
  yield (i, value*vec[j])
 
def REDUCE(i:int, products:Iterator[NamedTuple]):
  sum = 0
  for p in products:
    sum += p
  yield (i, sum)

def RECORDREADER():
  for i in range(mat.shape[0]):
    for j in range(mat.shape[1]):
      yield ((i, j), mat[i,j])
      
output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(0, 2.9874200657100816),
 (1, 2.9874200657100816),
 (2, 2.9874200657100816),
 (3, 2.9874200657100816),
 (4, 2.9874200657100816)]

## Inverted index 

In [48]:
from typing import Iterator

d1 = "it is what it is"
d2 = "what is it"
d3 = "it is a banana"
documents = [d1, d2, d3]

def RECORDREADER():
  for (docid, document) in enumerate(documents):
    yield ("{}".format(docid), document)
      
def MAP(docId:str, body:str):
  for word in set(body.split(' ')):
    yield (word, docId)
 
def REDUCE(word:str, docIds:Iterator[str]):
  yield (word, sorted(docIds))

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('what', ['0', '1']),
 ('is', ['0', '1', '2']),
 ('it', ['0', '1', '2']),
 ('banana', ['2']),
 ('a', ['2'])]

## WordCount

In [49]:
from typing import Iterator

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3]

def RECORDREADER():
  for (docid, document) in enumerate(documents):
    for (lineid, line) in enumerate(document.split('\n')):
      yield ("{}:{}".format(docid,lineid), line)

def MAP(docId:str, line:str):
  for word in line.split(" "):  
    yield (word, 1)
 
def REDUCE(word:str, counts:Iterator[int]):
  sum = 0
  for c in counts:
    sum += c
  yield (word, sum)

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('', 3), ('it', 9), ('is', 9), ('what', 5), ('a', 1), ('banana', 1)]

# MapReduce Distributed

Добавляется в модель фабрика RECORDREARER-ов --- INPUTFORMAT, функция распределения промежуточных результатов по партициям PARTITIONER, и функция COMBINER для частичной аггрегации промежуточных результатов до распределения по новым партициям.

In [50]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()
      
def groupbykey_distributed(map_partitions, PARTITIONER):
  global reducers
  partitions = [dict() for _ in range(reducers)]
  for map_partition in map_partitions:
    for (k2, v2) in map_partition:
      p = partitions[PARTITIONER(k2)]
      p[k2] = p.get(k2, []) + [v2]
  return [(partition_id, sorted(partition.items(), key=lambda x: x[0])) for (partition_id, partition) in enumerate(partitions)]
 
def PARTITIONER(obj):
  global reducers
  return hash(obj) % reducers
  
def MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, PARTITIONER=PARTITIONER, COMBINER=None):
  map_partitions = map(lambda record_reader: flatten(map(lambda k1v1: MAP(*k1v1), record_reader)), INPUTFORMAT())
  if COMBINER != None:
    map_partitions = map(lambda map_partition: flatten(map(lambda k2v2: COMBINER(*k2v2), groupbykey(map_partition))), map_partitions)
  reduce_partitions = groupbykey_distributed(map_partitions, PARTITIONER) # shuffle
  reduce_outputs = map(lambda reduce_partition: (reduce_partition[0], flatten(map(lambda reduce_input_group: REDUCE(*reduce_input_group), reduce_partition[1]))), reduce_partitions)
  
  print("{} key-value pairs were sent over a network.".format(sum([len(vs) for (k,vs) in flatten([partition for (partition_id, partition) in reduce_partitions])])))
  return reduce_outputs

## Спецификация MapReduce Distributed


```
f (k1, v1) -> (k2,v2)*
g (k2, v2*) -> (k3,v3)*
 
e1 (k1, v1)
e2 (k2, v2)
partition1 (k2, v2)*
partition2 (k2, v2*)*
 
flatmap (e1->e2*, e1*) -> partition1*
groupby (partition1*) -> partition2*

mapreduce ((k1,v1)*) -> (k3,v3)*
mapreduce .flatmap(f).groupby(k2).flatmap(g)
```



## WordCount 

In [51]:
from typing import Iterator
import numpy as np

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3, d1, d2, d3]

maps = 3
reducers = 2

def INPUTFORMAT():
  global maps
  
  def RECORDREADER(split):
    for (docid, document) in enumerate(split):
      for (lineid, line) in enumerate(document.split('\n')):
        yield ("{}:{}".format(docid,lineid), line)
      
  split_size =  int(np.ceil(len(documents)/maps))
  for i in range(0, len(documents), split_size):
    yield RECORDREADER(documents[i:i+split_size])

def MAP(docId:str, line:str):
  for word in line.split(" "):  
    yield (word, 1)
 
def REDUCE(word:str, counts:Iterator[int]):
  sum = 0
  for c in counts:
    sum += c
  yield (word, sum)
  
# try to set COMBINER=REDUCER and look at the number of values sent over the network 
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None) 
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

56 key-value pairs were sent over a network.


[(0, [('', 6), ('a', 2), ('banana', 2), ('what', 10)]),
 (1, [('is', 18), ('it', 18)])]

## TeraSort

In [52]:
import numpy as np

input_values = np.random.rand(30)
maps = 3
reducers = 2
min_value = 0.0
max_value = 1.0

def INPUTFORMAT():
  global maps
  
  def RECORDREADER(split):
    for value in split:
        yield (value, None)
      
  split_size =  int(np.ceil(len(input_values)/maps))
  for i in range(0, len(input_values), split_size):
    yield RECORDREADER(input_values[i:i+split_size])
    
def MAP(value:int, _):
  yield (value, None)
  
def PARTITIONER(key):
  global reducers
  global max_value
  global min_value
  bucket_size = (max_value-min_value)/reducers
  bucket_id = 0
  while((key>(bucket_id+1)*bucket_size) and ((bucket_id+1)*bucket_size<max_value)):
    bucket_id += 1
  return bucket_id

def REDUCE(value:int, _):
  yield (None,value)
  
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None, PARTITIONER=PARTITIONER)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

30 key-value pairs were sent over a network.


[(0,
  [(None, 0.004664326848199729),
   (None, 0.027116440619612048),
   (None, 0.08419354246006427),
   (None, 0.15530134662124506),
   (None, 0.198260254433421),
   (None, 0.23369628036631873),
   (None, 0.23549943133263318),
   (None, 0.24051685772335374),
   (None, 0.24891982439870597),
   (None, 0.26071223154111656),
   (None, 0.3005688241329053),
   (None, 0.32975540373260037),
   (None, 0.3900938777033659),
   (None, 0.3949904088774151),
   (None, 0.399177938812597),
   (None, 0.4052423393646022),
   (None, 0.44851910005253637)]),
 (1,
  [(None, 0.5528939730186209),
   (None, 0.5704863301652405),
   (None, 0.5899008940965315),
   (None, 0.6166221253164378),
   (None, 0.658098515185066),
   (None, 0.6779699424873804),
   (None, 0.7079908109679688),
   (None, 0.7239949380104916),
   (None, 0.7257466968989527),
   (None, 0.7451820300198185),
   (None, 0.7993057144122588),
   (None, 0.8374091582409976),
   (None, 0.9636688032102384)])]

# Упражнения
Упражнения взяты из Rajaraman A., Ullman J. D. Mining of massive datasets. – Cambridge University Press, 2011.


Для выполнения заданий переопределите функции RECORDREADER, MAP, REDUCE. Для модели распределённой системы может потребоваться переопределение функций PARTITION и COMBINER.

### Максимальное значение ряда

Разработайте MapReduce алгоритм, который находит максимальное число входного списка чисел.

In [53]:
import random

def MAP(chunk):
    return max(chunk)

def REDUCE(results):
    return max(results)

def RECORDREADER(count):
    return [random.randint(0, 100) for i in range(count)]

record = RECORDREADER(100)
print(record,'\n')

part_count = 5
record_partitional = [record[d:d + part_count] for d in range(0, len(record), part_count)]
print(record_partitional,'\n')

print(REDUCE(list(map(lambda x: MAP(x), record_partitional))))
     

[67, 36, 7, 88, 36, 79, 2, 18, 50, 29, 0, 47, 0, 58, 8, 100, 63, 93, 48, 90, 88, 25, 10, 44, 73, 80, 3, 61, 58, 7, 93, 48, 80, 17, 42, 9, 9, 20, 92, 8, 80, 34, 11, 21, 79, 27, 97, 24, 98, 61, 45, 72, 16, 13, 55, 62, 83, 47, 85, 19, 42, 57, 46, 27, 0, 48, 99, 17, 75, 4, 48, 85, 65, 84, 3, 96, 72, 8, 85, 96, 21, 41, 54, 33, 28, 58, 18, 29, 16, 52, 78, 86, 48, 15, 94, 91, 39, 8, 42, 65] 

[[67, 36, 7, 88, 36], [79, 2, 18, 50, 29], [0, 47, 0, 58, 8], [100, 63, 93, 48, 90], [88, 25, 10, 44, 73], [80, 3, 61, 58, 7], [93, 48, 80, 17, 42], [9, 9, 20, 92, 8], [80, 34, 11, 21, 79], [27, 97, 24, 98, 61], [45, 72, 16, 13, 55], [62, 83, 47, 85, 19], [42, 57, 46, 27, 0], [48, 99, 17, 75, 4], [48, 85, 65, 84, 3], [96, 72, 8, 85, 96], [21, 41, 54, 33, 28], [58, 18, 29, 16, 52], [78, 86, 48, 15, 94], [91, 39, 8, 42, 65]] 

100


### Арифметическое среднее

Разработайте MapReduce алгоритм, который находит арифметическое среднее.

$$\overline{X} = \frac{1}{n}\sum_{i=0}^{n} x_i$$


In [54]:
from multiprocessing import Pool

def MAP(chunk):
    return (1, chunk)

def REDUCE(key, nums: Iterator[NamedTuple]):
    sum = 0
    count = 0
    for n in nums:
        sum += n
        count += 1
    yield sum / count

def RECORDREADER(count):
    return [random.randint(0, 100) for i in range(count)]


map_output = list(map(lambda x: MAP(x), RECORDREADER(100)))
print(map_output)

shuffle_output = groupbykey(map_output)
shuffle_output = list(shuffle_output)

print(shuffle_output)

reduce_output = list(flatten(map(lambda x: REDUCE(*x), shuffle_output)))
print(reduce_output)


[(1, 40), (1, 17), (1, 14), (1, 55), (1, 41), (1, 48), (1, 5), (1, 87), (1, 40), (1, 50), (1, 87), (1, 0), (1, 51), (1, 28), (1, 93), (1, 98), (1, 22), (1, 48), (1, 67), (1, 44), (1, 44), (1, 0), (1, 67), (1, 31), (1, 56), (1, 6), (1, 5), (1, 5), (1, 53), (1, 80), (1, 18), (1, 15), (1, 75), (1, 50), (1, 37), (1, 73), (1, 94), (1, 16), (1, 99), (1, 0), (1, 17), (1, 6), (1, 16), (1, 2), (1, 8), (1, 94), (1, 7), (1, 74), (1, 74), (1, 83), (1, 4), (1, 49), (1, 78), (1, 48), (1, 58), (1, 12), (1, 59), (1, 18), (1, 69), (1, 48), (1, 100), (1, 93), (1, 22), (1, 53), (1, 42), (1, 41), (1, 50), (1, 79), (1, 72), (1, 63), (1, 75), (1, 55), (1, 61), (1, 89), (1, 57), (1, 22), (1, 52), (1, 12), (1, 80), (1, 73), (1, 45), (1, 8), (1, 87), (1, 76), (1, 70), (1, 86), (1, 8), (1, 98), (1, 54), (1, 83), (1, 4), (1, 76), (1, 98), (1, 40), (1, 50), (1, 90), (1, 3), (1, 99), (1, 69), (1, 44)]
[(1, [40, 17, 14, 55, 41, 48, 5, 87, 40, 50, 87, 0, 51, 28, 93, 98, 22, 48, 67, 44, 44, 0, 67, 31, 56, 6, 5, 5, 53

### GroupByKey на основе сортировки

Реализуйте groupByKey на основе сортировки, проверьте его работу на примерах

In [55]:

def group_by_key(iterable):
    sorted_data = sorted(iterable, key=lambda x: x[0])
    grouped_data = {}
    for key, value in sorted_data:
        if key not in grouped_data:
            grouped_data[key] = [value]
        else:
            grouped_data[key].append(value)
    result = []
    for key, value in grouped_data.items():
        result.append((key, value))
    return result

def MAP(num):
    return (1, num)


def REDUCE(key, nums: Iterator[NamedTuple]):
    sum = 0
    count = 0
    for n in nums:
        sum += n
        count += 1
    yield sum / count


def RECORDREADER(count):
    return [random.randint(0, 100) for i in range(count)]

map_output = list(map(lambda x: MAP(x), RECORDREADER(100)))
print(map_output)

shuffle_output = group_by_key(map_output)
shuffle_output = list(shuffle_output)

print(shuffle_output)

reduce_output = list(flatten(map(lambda x: REDUCE(*x), shuffle_output)))
print(reduce_output)
     

[(1, 55), (1, 100), (1, 0), (1, 17), (1, 24), (1, 1), (1, 45), (1, 93), (1, 61), (1, 74), (1, 26), (1, 34), (1, 37), (1, 97), (1, 62), (1, 87), (1, 52), (1, 25), (1, 86), (1, 33), (1, 37), (1, 11), (1, 33), (1, 30), (1, 31), (1, 76), (1, 87), (1, 35), (1, 54), (1, 80), (1, 34), (1, 14), (1, 49), (1, 13), (1, 92), (1, 10), (1, 34), (1, 26), (1, 6), (1, 43), (1, 33), (1, 57), (1, 80), (1, 77), (1, 91), (1, 40), (1, 95), (1, 49), (1, 65), (1, 96), (1, 38), (1, 11), (1, 64), (1, 60), (1, 31), (1, 17), (1, 12), (1, 96), (1, 32), (1, 55), (1, 13), (1, 68), (1, 15), (1, 83), (1, 98), (1, 92), (1, 88), (1, 83), (1, 48), (1, 25), (1, 14), (1, 2), (1, 45), (1, 16), (1, 9), (1, 19), (1, 14), (1, 44), (1, 45), (1, 55), (1, 99), (1, 33), (1, 2), (1, 12), (1, 2), (1, 66), (1, 22), (1, 71), (1, 45), (1, 95), (1, 78), (1, 40), (1, 14), (1, 71), (1, 91), (1, 0), (1, 30), (1, 69), (1, 97), (1, 27)]
[(1, [55, 100, 0, 17, 24, 1, 45, 93, 61, 74, 26, 34, 37, 97, 62, 87, 52, 25, 86, 33, 37, 11, 33, 30, 31, 7

### Drop duplicates (set construction, unique elements, distinct)

Реализуйте распределённую операцию исключения дубликатов

In [56]:
input_collection = [
    User(id=0, age=54, gender="male", social_contacts=20),
    User(id=0, age=25, gender="female", social_contacts=240),
    User(id=1, age=27, gender="male", social_contacts=642),
    User(id=2, age=16, gender="male", social_contacts=123),
    User(id=2, age=35, gender="female", social_contacts=247),
    User(id=3, age=31, gender="male", social_contacts=521),
    User(id=3, age=32, gender="female", social_contacts=753),
]

def flatten(nested_iterable):
    for iterable in nested_iterable:
        for element in iterable:
            yield element


def groupbykey(iterable):
    t = {}
    for k2, v2 in iterable:
        t[k2] = t.get(k2, []) + [v2]
    return t.items()


def groupbykey_distributed(map_partitions, PARTITIONER):
    global reducers
    partitions = [dict() for _ in range(reducers)]
    for map_partition in map_partitions:
        for k2, v2 in map_partition:
            p = partitions[PARTITIONER(k2)]
            p[k2] = p.get(k2, []) + [v2]
    return [
        (partition_id, sorted(partition.items(), key=lambda x: x[0]))
        for (partition_id, partition) in enumerate(partitions)
    ]


def PARTITIONER(obj):
    global reducers
    return hash(obj) % reducers


def MapReduceDistributed(
    INPUTFORMAT, MAP, REDUCE, PARTITIONER=PARTITIONER, COMBINER=None
):
    map_partitions = map(
        lambda record_reader: flatten(map(lambda k1v1: MAP(*k1v1), record_reader)),
        INPUTFORMAT(),
    )
    if COMBINER != None:
        map_partitions = map(
            lambda map_partition: flatten(
                map(lambda k2v2: COMBINER(*k2v2), groupbykey(map_partition))
            ),
            map_partitions,
        )
    reduce_partitions = groupbykey_distributed(map_partitions, PARTITIONER)  # shuffle
    reduce_outputs = map(
        lambda reduce_partition: (
            reduce_partition[0],
            flatten(
                map(
                    lambda reduce_input_group: REDUCE(*reduce_input_group),
                    reduce_partition[1],
                )
            ),
        ),
        reduce_partitions,
    )

    print(
        "{} key-value pairs were sent over a network.".format(
            sum(
                [
                    len(vs)
                    for (k, vs) in flatten(
                        [partition for (partition_id, partition) in reduce_partitions]
                    )
                ]
            )
        )
    )
    return reduce_outputs

maps = 3
reducers = 2


def INPUTFORMAT():
    global maps

    def RECORDREADER(split):
        for userindex, user in enumerate(split):
            yield (userindex, user)

    # Разделение массива пользователей между вычислительными узлами
    split_size = int(np.ceil(len(input_collection) / maps))
    for i in range(0, len(input_collection), split_size):
        yield RECORDREADER(input_collection[i : i + split_size])


# Группировка пользователей по id
def MAP_UNIQUE(_, user: str):
    yield (user.id, user)


# Из множества пользователей с одинаковым id остается только первый
def REDUCE_UNIQUE(user_id: int, users: Iterator[int]):
    yield users[0]


# try to set COMBINER=REDUCER and look at the number of values sent over the network
partitioned_output = MapReduceDistributed(
    INPUTFORMAT, MAP_UNIQUE, REDUCE_UNIQUE, COMBINER=None
)
partitioned_output = [
    (partition_id, list(partition)) for (partition_id, partition) in partitioned_output
]
partitioned_output

7 key-value pairs were sent over a network.


[(0,
  [User(id=0, age=54, social_contacts=20, gender='male'),
   User(id=2, age=16, social_contacts=123, gender='male')]),
 (1,
  [User(id=1, age=27, social_contacts=642, gender='male'),
   User(id=3, age=31, social_contacts=521, gender='male')])]

#Операторы реляционной алгебры
### Selection (Выборка)

**The Map Function**: Для  каждого кортежа $t \in R$ вычисляется истинность предиката $C$. В случае истины создаётся пара ключ-значение $(t, t)$. В паре ключ и значение одинаковы, равны $t$.

**The Reduce Function:** Роль функции Reduce выполняет функция идентичности, которая возвращает то же значение, что получила на вход.



In [57]:
from collections import defaultdict

def MAP(el_list):
    mapped_result = defaultdict(list)
    for t in el_list:
        if C(t):  # Предикат C
            mapped_result[t].append(t)
    return mapped_result.items()


def REDUCE(mapped_items):
    reduced_result = []
    print(mapped_items)
    for values in mapped_items:
        for value in values:
            reduced_result.append(value[0])
    return reduced_result

def C(t):
    return t[0] % 2 == 0  # Выбираем только кортежи с четным первым элементом


def RECORDREADER(count):
    return [(random.randint(0, 100), random.randint(0, 100), random.randint(0, 100)) for i in range(count)]

record = RECORDREADER(100)
print(record)

part_count = 5
record_partitional = [record[d:d + part_count] for d in range(0, len(record), part_count)]
print(record_partitional)

print(REDUCE(list(map(lambda x: MAP(x), record_partitional))))

[(84, 10, 52), (2, 96, 27), (19, 45, 18), (46, 33, 66), (52, 41, 91), (95, 17, 49), (64, 12, 59), (46, 5, 57), (9, 86, 20), (41, 100, 16), (59, 27, 90), (34, 22, 13), (47, 69, 40), (5, 49, 92), (22, 40, 38), (79, 80, 29), (94, 21, 17), (76, 4, 48), (48, 37, 22), (25, 25, 99), (32, 6, 58), (1, 11, 89), (48, 3, 33), (69, 9, 9), (74, 67, 41), (12, 53, 57), (87, 60, 18), (63, 58, 96), (16, 49, 2), (58, 24, 93), (60, 50, 65), (79, 2, 66), (40, 34, 5), (9, 40, 2), (95, 26, 20), (28, 45, 32), (80, 37, 20), (0, 28, 68), (14, 10, 73), (53, 88, 36), (42, 45, 59), (15, 35, 63), (85, 97, 17), (36, 19, 11), (46, 28, 14), (10, 53, 66), (99, 52, 25), (74, 88, 43), (70, 84, 19), (23, 100, 56), (13, 32, 74), (51, 42, 22), (27, 18, 10), (39, 83, 60), (95, 6, 20), (16, 37, 65), (97, 91, 97), (40, 93, 27), (60, 55, 88), (5, 94, 11), (61, 5, 29), (76, 40, 37), (30, 45, 13), (77, 25, 27), (63, 96, 25), (63, 84, 69), (69, 52, 14), (67, 53, 24), (72, 74, 31), (34, 13, 34), (65, 23, 82), (83, 27, 62), (32, 88,

### Projection (Проекция)

Проекция на множество атрибутов $S$.

**The Map Function:** Для каждого кортежа $t \in R$ создайте кортеж $t′$, исключая  из $t$ те значения, атрибуты которых не принадлежат  $S$. Верните пару $(t′, t′)$.

**The Reduce Function:** Для каждого ключа $t′$, созданного любой Map задачей, вы получаете одну или несколько пар $(t′, t′)$. Reduce функция преобразует $(t′, [t′, t′, . . . , t′])$ в $(t′, t′)$, так, что для ключа $t′$ возвращается одна пара  $(t′, t′)$.

In [58]:
S = set([1 ,12, 56])

def MAP(t):
  res_list = []
  for el in t:
    if el in S:
      res_list.append(el)
    res = tuple(res_list)
    return (res, res)


def REDUCE(key, values: Iterator[NamedTuple]):
    return (key, key)

def RECORDREADER(count):
    return [(random.randint(0, 100), random.randint(0, 100), random.randint(0, 100)) for i in range(count)]

def group_by_key(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

record = RECORDREADER(100)
print(record)
map_output = list(map(lambda x: MAP(x), RECORDREADER(100)))
print(map_output)

shuffle_output = group_by_key(map_output)
shuffle_output = list(shuffle_output)

print(shuffle_output)

reduce_output = list(map(lambda x: REDUCE(*x)[0], shuffle_output))
print(reduce_output)
     

[(92, 20, 42), (94, 14, 34), (30, 56, 61), (36, 13, 26), (99, 19, 31), (4, 70, 4), (1, 44, 15), (6, 50, 33), (4, 33, 76), (42, 2, 77), (8, 38, 44), (63, 75, 40), (58, 22, 28), (30, 89, 42), (25, 38, 5), (29, 85, 40), (92, 77, 58), (89, 73, 70), (4, 79, 27), (23, 94, 1), (70, 24, 14), (91, 65, 8), (63, 100, 0), (46, 51, 52), (74, 45, 74), (20, 59, 11), (45, 54, 86), (82, 39, 56), (59, 11, 85), (53, 5, 76), (27, 96, 75), (27, 94, 12), (14, 55, 15), (64, 39, 93), (25, 29, 33), (80, 14, 84), (60, 99, 84), (67, 75, 39), (85, 3, 1), (46, 87, 28), (38, 32, 17), (82, 47, 11), (85, 23, 94), (67, 10, 84), (47, 44, 15), (40, 76, 15), (97, 34, 39), (13, 48, 76), (85, 45, 15), (70, 67, 12), (28, 43, 65), (58, 75, 85), (9, 37, 75), (91, 7, 86), (51, 52, 70), (0, 7, 32), (4, 40, 92), (48, 55, 92), (79, 10, 10), (49, 80, 12), (78, 56, 53), (41, 90, 46), (36, 79, 76), (23, 26, 74), (11, 32, 38), (26, 99, 28), (30, 43, 92), (40, 80, 97), (87, 25, 16), (81, 67, 2), (41, 48, 11), (85, 89, 74), (29, 22, 96

### Union (Объединение)

**The Map Function:** Превратите каждый входной кортеж $t$ в пару ключ-значение $(t, t)$.

**The Reduce Function:** С каждым ключом $t$ будет ассоциировано одно или два значений. В обоих случаях создайте $(t, t)$ в качестве выходного значения.

In [59]:
def MAP(t):
  return (t, t)


def REDUCE(key, values: Iterator[NamedTuple]):
    return (key, key)

def RECORDREADER(count):
    return [(random.randint(0, 100), random.randint(0, 100), random.randint(0, 100)) for i in range(count)]

def group_by_key(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

record = RECORDREADER(100)
print(record)
map_output = list(map(lambda x: MAP(x), RECORDREADER(100)))
print(map_output)

shuffle_output = group_by_key(map_output)
shuffle_output = list(shuffle_output)

print(shuffle_output)

reduce_output = list(map(lambda x: REDUCE(*x)[0], shuffle_output))
print(reduce_output)

[(52, 72, 6), (75, 5, 44), (9, 12, 20), (73, 83, 94), (46, 38, 78), (0, 77, 88), (0, 10, 90), (60, 63, 44), (40, 73, 50), (58, 90, 63), (20, 75, 5), (53, 69, 30), (27, 29, 91), (78, 98, 32), (46, 40, 39), (88, 51, 63), (62, 57, 65), (75, 21, 67), (46, 81, 91), (78, 68, 19), (19, 38, 75), (53, 50, 29), (86, 100, 60), (55, 28, 92), (63, 18, 28), (52, 6, 49), (38, 56, 83), (22, 13, 60), (76, 23, 5), (83, 74, 49), (3, 25, 59), (37, 58, 1), (7, 47, 86), (37, 72, 60), (30, 51, 84), (43, 100, 74), (58, 12, 39), (14, 31, 48), (93, 15, 16), (77, 44, 46), (34, 64, 58), (73, 43, 22), (49, 65, 18), (85, 48, 0), (41, 14, 31), (89, 25, 69), (95, 68, 17), (86, 10, 37), (68, 42, 90), (67, 80, 46), (100, 61, 78), (43, 13, 92), (78, 61, 88), (16, 17, 21), (70, 8, 57), (38, 27, 10), (74, 24, 79), (18, 21, 75), (96, 36, 90), (76, 36, 4), (8, 95, 91), (94, 7, 71), (61, 51, 59), (44, 47, 53), (16, 54, 80), (76, 87, 82), (1, 18, 99), (71, 69, 91), (44, 46, 47), (34, 40, 34), (59, 86, 9), (88, 20, 68), (67, 5

### Intersection (Пересечение)

**The Map Function:** Превратите каждый кортеж $t$ в пары ключ-значение $(t, t)$.

**The Reduce Function:** Если для ключа $t$ есть список из двух элементов $[t, t]$ $-$ создайте пару $(t, t)$. Иначе, ничего не создавайте.

In [60]:
def MAP(t):
  return (t, t)


def REDUCE(key, values: Iterator[NamedTuple]):
  if len(values) == 2:
    return (key, key)

def RECORDREADER(count):
    return [(random.randint(0, 3), random.randint(0, 3)) for i in range(count)]

def group_by_key(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

record = RECORDREADER(100)
print(record)
map_output = list(map(lambda x: MAP(x), RECORDREADER(100)))
print(map_output)

shuffle_output = group_by_key(map_output)
shuffle_output = list(shuffle_output)

print(shuffle_output)

reduce_output = [el[0] for el in list(map(lambda x: REDUCE(*x), shuffle_output)) if el is not None]
print(reduce_output)

[(0, 2), (3, 1), (0, 3), (2, 3), (3, 2), (3, 1), (0, 1), (0, 1), (2, 0), (2, 3), (1, 3), (1, 3), (1, 0), (3, 2), (3, 0), (1, 3), (1, 3), (0, 2), (0, 1), (2, 1), (0, 2), (0, 1), (2, 2), (0, 1), (0, 3), (0, 0), (1, 1), (0, 0), (3, 3), (3, 1), (0, 2), (0, 0), (3, 1), (2, 1), (1, 2), (1, 3), (0, 0), (0, 2), (1, 1), (2, 0), (3, 2), (1, 1), (1, 0), (2, 3), (3, 3), (2, 0), (3, 3), (2, 0), (2, 2), (3, 2), (3, 1), (2, 3), (3, 2), (3, 1), (2, 2), (1, 2), (2, 0), (3, 0), (1, 1), (2, 1), (3, 0), (0, 0), (2, 2), (1, 1), (0, 0), (0, 1), (0, 3), (1, 3), (3, 0), (1, 0), (0, 3), (3, 0), (2, 1), (2, 0), (0, 0), (0, 1), (3, 2), (1, 0), (1, 1), (3, 1), (3, 1), (1, 3), (1, 2), (1, 2), (0, 1), (3, 0), (1, 3), (2, 0), (3, 2), (0, 0), (2, 1), (3, 3), (2, 3), (3, 0), (2, 3), (3, 3), (3, 0), (3, 3), (3, 1), (2, 2)]
[((1, 2), (1, 2)), ((3, 0), (3, 0)), ((3, 3), (3, 3)), ((1, 1), (1, 1)), ((2, 3), (2, 3)), ((1, 3), (1, 3)), ((2, 3), (2, 3)), ((0, 2), (0, 2)), ((2, 2), (2, 2)), ((2, 1), (2, 1)), ((2, 0), (2, 0)), 

### Difference (Разница)

**The Map Function:** Для кортежа $t \in R$, создайте пару $(t, R)$, и для кортежа $t \in S$, создайте пару $(t, S)$. Задумка заключается в том, чтобы значение пары было именем отношения $R$ or $S$, которому принадлежит кортеж (а лучше, единичный бит, по которому можно два отношения различить $R$ or $S$), а не весь набор атрибутов отношения.

**The Reduce Function:** Для каждого ключа $t$, если соответствующее значение является списком $[R]$, создайте пару $(t, t)$. В иных случаях не предпринимайте действий.

In [61]:
rels = [1, 2]
class Tuple: # класс кортежа, который хранит данные и имя представления из массива rels выше
  def __init__(self, data: tuple, rel_id: int):
    self.data = data
    self.rel_id = rel_id

def get_random_tuple(count):
  data = tuple([(random.randint(0, 3), random.randint(0, 3)) for i in range(count)])
  rel_id = rels[random.randint(0, len(rels) - 1)]
  return Tuple(data, rel_id)


def RECORDREADER(count):
    return [get_random_tuple(3) for i in range(count)]

def MAP(t: Tuple):
  return (t.data, t.rel_id)

def REDUCE(key, values: Iterator[NamedTuple]):
  if values == [rels[0]]: # если соответствующее значение является списком  [R] , создайте пару  (t,t)
    return (key, key)

def group_by_key(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

record = RECORDREADER(100)
print(record)
map_output = list(map(lambda x: MAP(x), RECORDREADER(100)))
print(map_output)

shuffle_output = group_by_key(map_output)
shuffle_output = list(shuffle_output)

print(shuffle_output)

reduce_output = [el[0] for el in list(map(lambda x: REDUCE(*x), shuffle_output)) if el is not None]
print(reduce_output)

[<__main__.Tuple object at 0x000001E806BA8410>, <__main__.Tuple object at 0x000001E81F8ABED0>, <__main__.Tuple object at 0x000001E81F8ABB90>, <__main__.Tuple object at 0x000001E81F8AB3D0>, <__main__.Tuple object at 0x000001E81F8AB690>, <__main__.Tuple object at 0x000001E81F81A790>, <__main__.Tuple object at 0x000001E81F81A850>, <__main__.Tuple object at 0x000001E81F818390>, <__main__.Tuple object at 0x000001E81F819310>, <__main__.Tuple object at 0x000001E81F81A810>, <__main__.Tuple object at 0x000001E81F819550>, <__main__.Tuple object at 0x000001E81F818350>, <__main__.Tuple object at 0x000001E81F87E710>, <__main__.Tuple object at 0x000001E81F87E490>, <__main__.Tuple object at 0x000001E81F87F650>, <__main__.Tuple object at 0x000001E81F87F190>, <__main__.Tuple object at 0x000001E81F87F310>, <__main__.Tuple object at 0x000001E81F87DD10>, <__main__.Tuple object at 0x000001E81F87E610>, <__main__.Tuple object at 0x000001E81F87DDD0>, <__main__.Tuple object at 0x000001E81F87E790>, <__main__.Tu

### Natural Join

**The Map Function:** Для каждого кортежа $(a, b)$ отношения $R$, создайте пару $(b,(R, a))$. Для каждого кортежа $(b, c)$ отношения $S$, создайте пару $(b,(S, c))$.

**The Reduce Function:** Каждый ключ $b$ будет асоциирован со списком пар, которые принимают форму либо $(R, a)$, либо $(S, c)$. Создайте все пары, одни, состоящие из  первого компонента $R$, а другие, из первого компонента $S$, то есть $(R, a)$ и $(S, c)$. На выходе вы получаете последовательность пар ключ-значение из списков ключей и значений. Ключ не нужен. Каждое значение, это тройка $(a, b, c)$ такая, что $(R, a)$ и $(S, c)$ это принадлежат входному списку значений.

In [62]:
rels = [1, 2]
class Tuple: # класс кортежа, который хранит данные и имя представления из массива rels выше
  def __init__(self, data: tuple, rel_id: int):
    self.data = data
    self.rel_id = rel_id

def get_random_tuple():
  data = (random.randint(0, 3), random.randint(0, 3))
  rel_id = rels[random.randint(0, len(rels) - 1)]
  return Tuple(data, rel_id)


def RECORDREADER(count):
  return [get_random_tuple() for i in range(count)]

def MAP(t: Tuple):
  if t.rel_id == rels[0]:
    return (t.data[1], (t.rel_id, t.data[0]))
  else:
    return (t.data[0], (t.rel_id, t.data[1]))

def REDUCE(key, values: Iterator[NamedTuple]):
  res = []
  for v in values:
    res.append((v[0], key, v[1]))
  return res

def group_by_key(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

record = RECORDREADER(100)
print(record)
map_output = list(map(lambda x: MAP(x), RECORDREADER(100)))
print(map_output)

shuffle_output = group_by_key(map_output)
shuffle_output = list(shuffle_output)

print(shuffle_output)

reduce_output = list(map(lambda x: REDUCE(*x), shuffle_output))
print(reduce_output)

[<__main__.Tuple object at 0x000001E81F819810>, <__main__.Tuple object at 0x000001E806BC0F90>, <__main__.Tuple object at 0x000001E81F819DD0>, <__main__.Tuple object at 0x000001E81F8C9ED0>, <__main__.Tuple object at 0x000001E81F8C8F90>, <__main__.Tuple object at 0x000001E81F8CAC90>, <__main__.Tuple object at 0x000001E81F8CAA90>, <__main__.Tuple object at 0x000001E81F8C9950>, <__main__.Tuple object at 0x000001E81F8C86D0>, <__main__.Tuple object at 0x000001E81F8C9BD0>, <__main__.Tuple object at 0x000001E81F8C8650>, <__main__.Tuple object at 0x000001E81F8C8610>, <__main__.Tuple object at 0x000001E81F8C8390>, <__main__.Tuple object at 0x000001E81F8C8450>, <__main__.Tuple object at 0x000001E81F8C84D0>, <__main__.Tuple object at 0x000001E81F8C8550>, <__main__.Tuple object at 0x000001E81F8C8590>, <__main__.Tuple object at 0x000001E81F8C8510>, <__main__.Tuple object at 0x000001E81F8C8490>, <__main__.Tuple object at 0x000001E81F8C8410>, <__main__.Tuple object at 0x000001E81F8C8350>, <__main__.Tu

### Grouping and Aggregation (Группировка и аггрегация)

**The Map Function:** Для каждого кортежа $(a, b, c$) создайте пару $(a, b)$.

**The Reduce Function:** Ключ представляет ту или иную группу. Примение аггрегирующую операцию $\theta$ к списку значений $[b1, b2, . . . , bn]$ ассоциированных с ключом $a$. Возвращайте в выходной поток $(a, x)$, где $x$ результат применения  $\theta$ к списку. Например, если $\theta$ это $SUM$, тогда $x = b1 + b2 + · · · + bn$, а если $\theta$ is $MAX$, тогда $x$ это максимальное из значений $b1, b2, . . . , bn$.

In [63]:
def get_random_tuple():
  return (random.randint(0, 3), random.randint(0, 3), random.randint(0, 3))

def RECORDREADER(count):
  return [get_random_tuple() for i in range(count)]

def MAP(t: Tuple):
  return (t[0], t[1])

def tetta(values):
  return sum(values)


def REDUCE(key, values: Iterator[NamedTuple]):
  x = tetta(values)
  return (key, x)

def group_by_key(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

record = RECORDREADER(100)
print(record)
map_output = list(map(lambda x: MAP(x), RECORDREADER(100)))
print(map_output)

shuffle_output = group_by_key(map_output)
shuffle_output = list(shuffle_output)

print(shuffle_output)

reduce_output = list(map(lambda x: REDUCE(*x), shuffle_output))
print(reduce_output)

[(0, 3, 1), (1, 2, 3), (1, 1, 2), (2, 2, 2), (0, 2, 0), (1, 2, 0), (0, 0, 1), (3, 0, 3), (2, 0, 0), (3, 1, 0), (0, 0, 0), (2, 1, 0), (0, 2, 2), (0, 2, 0), (0, 2, 3), (2, 2, 3), (1, 0, 1), (3, 0, 0), (0, 0, 2), (0, 0, 2), (0, 3, 2), (1, 0, 2), (0, 0, 1), (2, 3, 3), (1, 0, 0), (0, 3, 0), (3, 2, 0), (0, 3, 3), (3, 2, 2), (1, 0, 1), (0, 1, 1), (2, 1, 2), (0, 1, 2), (2, 2, 1), (2, 1, 1), (0, 2, 0), (0, 0, 2), (0, 0, 1), (3, 1, 2), (3, 1, 2), (3, 2, 0), (3, 1, 0), (2, 0, 2), (3, 2, 3), (2, 0, 3), (0, 3, 0), (3, 3, 1), (1, 2, 0), (0, 1, 3), (2, 1, 3), (3, 3, 3), (3, 0, 1), (0, 0, 1), (3, 0, 3), (0, 0, 1), (2, 0, 1), (3, 1, 3), (1, 2, 0), (1, 2, 2), (2, 1, 2), (1, 3, 3), (2, 1, 2), (2, 1, 0), (2, 1, 1), (2, 1, 0), (1, 1, 1), (0, 2, 1), (1, 3, 0), (3, 2, 1), (2, 1, 0), (3, 0, 2), (1, 1, 3), (0, 1, 0), (3, 3, 1), (2, 1, 0), (3, 3, 3), (3, 1, 0), (3, 1, 0), (3, 1, 0), (3, 3, 3), (3, 0, 3), (1, 1, 0), (1, 2, 1), (1, 3, 1), (3, 0, 0), (2, 2, 1), (3, 1, 0), (1, 2, 2), (1, 1, 2), (1, 0, 1), (3, 3, 0)

# 

### Matrix-Vector multiplication

Случай, когда вектор не помещается в памяти Map задачи


## Matrix multiplication (Перемножение матриц)

Если у нас есть матрица $M$ с элементами $m_{ij}$ в строке $i$ и столбце $j$, и матрица $N$ с элементами $n_{jk}$ в строке $j$ и столбце $k$, тогда их произведение $P = MN$ есть матрица $P$ с элементами $p_{ik}$ в строке $i$ и столбце $k$, где

$$p_{ik} =\sum_{j} m_{ij}n_{jk}$$

Необходимым требованием является одинаковое количество столбцов в $M$ и строк в $N$, чтобы операция суммирования по  $j$ была осмысленной. Мы можем размышлять о матрице, как об отношении с тремя атрибутами: номер строки, номер столбца, само значение. Таким образом матрица $M$ предстваляется как отношение $ M(I, J, V )$, с кортежами $(i, j, m_{ij})$, и, аналогично, матрица $N$ представляется как отношение $N(J, K, W)$, с кортежами $(j, k, n_{jk})$. Так как большие матрицы как правило разреженные (большинство значений равно 0), и так как мы можем нулевыми значениями пренебречь (не хранить), такое реляционное представление достаточно эффективно для больших матриц. Однако, возможно, что координаты $i$, $j$, и $k$ неявно закодированы в смещение позиции элемента относительно начала файла, вместо явного хранения. Тогда, функция Map (или Reader) должна быть разработана таким образом, чтобы реконструировать компоненты $I$, $J$, и $K$ кортежей из смещения.

Произведение $MN$ это фактически join, за которым следуют группировка по ключу и аггрегация. Таким образом join отношений $M(I, J, V )$ и $N(J, K, W)$, имеющих общим только атрибут $J$, создаст кортежи $(i, j, k, v, w)$ из каждого кортежа $(i, j, v) \in M$ и кортежа $(j, k, w) \in N$. Такой 5 компонентный кортеж представляет пару элементов матрицы $(m_{ij} , n_{jk})$. Что нам хотелось бы получить на самом деле, это произведение этих элементов, то есть, 4 компонентный кортеж$(i, j, k, v \times w)$, так как он представляет произведение $m_{ij}n_{jk}$. Мы представляем отношение как результат одной MapReduce операции, в которой мы можем произвести группировку и аггрегацию, с $I$ и $K$  атрибутами, по которым идёт группировка, и суммой  $V \times W$. 





In [64]:
# MapReduce model
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
  return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

Реализуйте перемножение матриц с использованием модельного кода MapReduce для одной машины в случае, когда одна матрица хранится в памяти, а другая генерируется RECORDREADER-ом.

In [65]:

import numpy as np
I = 2
J = 3
K = 4*10
small_mat = np.random.rand(I,J) # it is legal to access this from RECORDREADER, MAP, REDUCE
big_mat = np.random.rand(J,K)

def RECORDREADER():
  for j in range(big_mat.shape[0]):
    for k in range(big_mat.shape[1]):
      yield ((j, k), big_mat[j,k])

def MAP(k1, v1):
  (j, k) = k1
  w = v1
  for i in range(small_mat.shape[0]):
    yield ((i, k), w * small_mat[i][j])

def REDUCE(key, values):
  (i, k) = key
  el_value = 0
  for v in values:
    el_value += v
  yield ((i, k), el_value)

Проверьте своё решение

In [66]:
# CHECK THE SOLUTION
reference_solution = np.matmul(small_mat, big_mat) 
solution = MapReduce(RECORDREADER, MAP, REDUCE)

def asmatrix(reduce_output):
  reduce_output = list(reduce_output)
  I = max(i for ((i,k), vw) in reduce_output)+1
  K = max(k for ((i,k), vw) in reduce_output)+1
  mat = np.empty(shape=(I,K))
  for ((i,k), vw) in reduce_output:
    mat[i,k] = vw
  return mat

np.allclose(reference_solution, asmatrix(solution)) # should return true

True

In [67]:
reduce_output = list(MapReduce(RECORDREADER, MAP, REDUCE))
max(i for ((i,k), vw) in reduce_output)

1

Реализуйте перемножение матриц  с использованием модельного кода MapReduce для одной машины в случае, когда обе матрицы генерируются в RECORDREADER. Например, сначала одна, а потом другая.

In [68]:
import numpy as np
I = 2
J = 3
K = 4*10
small_mat = np.random.rand(I,J)
big_mat = np.random.rand(J,K)
reference_solution = np.matmul(small_mat, big_mat)

def RECORDREADER():
  for i in range(small_mat.shape[0]):
    for j in range(small_mat.shape[1]):
      yield ((0, i, j), small_mat[i,j]) # первая матрица

  for j in range(big_mat.shape[0]):
    for k in range(big_mat.shape[1]):
      yield ((1, j, k), big_mat[j,k]) # вторая матрица

def MAP_JOIN(k1, v1):
  (mat_num, i, j) = k1
  w = v1
  if mat_num == 0:
    yield (j, (mat_num, i, w))
  else:
    yield (i, (mat_num, j, w))


def REDUCE_JOIN(key, values):
  from_first_mat = [v for v in values if v[0] == 0]
  from_second_mat = [v for v in values if v[0] == 1]
  for f in from_first_mat:
    for s in from_second_mat:
      yield ((f[1], s[1]), f[2] * s[2])


def MAP_MUL(k1, v1):
  (i, k) = k1
  yield ((i, k), v1)


def REDUCE_MUL(key, values):
  res_el_value = 0
  for v in values:
    res_el_value += v
  yield (key, res_el_value)


def GET_JOINED():
  for j in joined:
    yield j

joined = MapReduce(RECORDREADER, MAP_JOIN, REDUCE_JOIN)
solution = MapReduce(GET_JOINED, MAP_MUL, REDUCE_MUL)
np.allclose(reference_solution, asmatrix(solution)) # should return true

True

Реализуйте перемножение матриц с использованием модельного кода MapReduce Distributed, когда каждая матрица генерируется в своём RECORDREADER. 

In [69]:
maps = 2
reducers = 2


def INPUTFORMAT():
    global maps

    # Так же, как и раньше, генерируются пары элементов из обоих матриц
    def RECORDREADER(i_range):
        for i in i_range:
            for j in range(J):
                for k in range(K):
                    yield (((i, j), small_mat[i, j]), ((j, k), big_mat[j, k]))

    # Разделение между вычислительными узлами происходит по строкам первой матрицы
    split_size = int(np.ceil(I / maps))
    for i in range(0, I, split_size):
        yield RECORDREADER(range(i, i + split_size))


# Возвращает произведения соответствующих элементов из обоих матриц, группируя их по индексу в результирующей матрице
def MAP(element1, element2):
    (i, j), v1 = element1
    (j, k), v2 = element2

    yield ((i, k), v1 * v2)


# Суммирует полученные произведения
def REDUCE(key, values):
    (i, k) = key

    # solution code that yield(k3,v3) pairs
    k3 = (i, k)

    v3 = 0
    for j in range(J):
        v3 += values[j]

    yield (k3, v3)


# try to set COMBINER=REDUCER and look at the number of values sent over the network
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None)
partitioned_output = [
    (partition_id, list(partition)) for (partition_id, partition) in partitioned_output
]

# Соединение полученных произведений с разных вычислительных узлов
solution = []
for output_part in partitioned_output:
    for element in output_part[1]:
        solution += [element]

print(asmatrix(solution))
np.allclose(reference_solution, asmatrix(solution))  # should return true

240 key-value pairs were sent over a network.
[[0.65865836 0.46910048 0.78355395 0.53554272 0.27875415 0.59131802
  0.24531517 0.87775698 0.67908995 0.58657848 0.32353139 0.10471164
  0.57846975 0.65680141 0.7286292  0.56115401 0.64288319 0.88498144
  0.77388339 0.70485128 0.11951504 0.10914563 0.8251213  0.63133299
  0.28963505 0.54815588 0.38681736 0.51183813 0.66725458 0.32372938
  0.54749907 0.18051246 0.93698647 0.87660546 0.71585815 0.59174882
  0.49985123 0.68750673 0.23421548 0.57175287]
 [0.74332642 0.49137643 0.77152082 0.55457369 0.27597359 0.65968644
  0.30422072 0.8744454  0.689904   0.61780447 0.34943198 0.13364368
  0.58318262 0.64887492 0.74532945 0.5882953  0.76609483 0.96513948
  0.82279681 0.68068651 0.12105031 0.11402273 0.88068795 0.67318379
  0.33671756 0.49439802 0.38744228 0.51597437 0.63817548 0.37657653
  0.56333815 0.21413891 0.92684016 0.86451021 0.78588529 0.62885953
  0.56066641 0.68234669 0.26063563 0.65066723]]


True

Обобщите предыдущее решение на случай, когда каждая матрица генерируется несколькими RECORDREADER-ами, и проверьте его работоспособность. Будет ли работать решение, если RECORDREADER-ы будут генерировать случайное подмножество элементов матрицы?

Решение будет работать, так как каждая пара элементов, генерируемая RECORDREADER, идентифицируется по ее индексам